<a href="https://colab.research.google.com/github/Josinaldo-Jr/VeganaAI/blob/main/VeganAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌱 Bem-vindo ao Vegan AI 🌱

## Sobre o Vegan AI
Vegan AI é uma aplicação que combina inteligência artificial com informações relevantes para pessoas veganas. Seja você um vegano de longa data ou alguém interessado em adotar um estilo de vida vegano, o Vegan AI está aqui para fornecer receitas, guias e dicas úteis para sua jornada vegana.

## Recursos Principais
- **Busca Inteligente**: Use nossa poderosa busca para encontrar informações relevantes sobre veganismo.
- **Conteúdo Veganizado**: Descubra receitas deliciosas, guias úteis e dicas essenciais para uma vida vegana saudável.

## Como Funciona
1. Insira sua consulta na barra de busca.
2. Receba um trecho relevante de texto sobre o tema.

## Experimente Agora!
Prepare-se para uma experiência vegana única com Vegan AI. Comece sua jornada vegana ou aprimore seu conhecimento com nossos recursos exclusivos!

In [3]:
# Instalação de Dependências
!pip install -U -q google-generativeai

In [8]:
# Importações e Configurações Iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai
# Configuração da Chave da API do Google
from google.colab import userdata
api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

In [10]:
# Definição dos Documentos para Veganos
DOCUMENT1 = {
    "Título": "Receita Vegana: Hambúrguer de Grão-de-Bico",
    "Conteúdo": "Experimente esta deliciosa receita vegana de hambúrguer de grão-de-bico! Para preparar, basta amassar grão-de-bico cozido com cebola picada, alho, aveia em flocos, páprica, cominho e sal. Molde a mistura em hambúrgueres e grelhe até ficarem dourados. Sirva em um pão integral com alface, tomate e molho vegano de sua escolha."
}

DOCUMENT2 = {
    "Título": "Guia Vegano: Alternativas Saborosas aos Produtos de Origem Animal",
    "Conteúdo": "Neste guia vegano, você encontrará alternativas deliciosas aos produtos de origem animal. Substitua o leite de vaca por leites vegetais como o leite de amêndoa, soja ou aveia. Para queijo, experimente opções à base de castanhas ou batatas. Em vez de carne, explore proteínas vegetais como o tofu, seitan ou lentilhas. Com essas alternativas, você pode desfrutar de refeições saborosas e nutritivas enquanto mantém seu compromisso com o veganismo."
}

DOCUMENT3 = {
    "Título": "Dicas para uma Dieta Vegana Equilibrada",
    "Conteúdo": "Seguir uma dieta vegana pode ser saudável e equilibrado com as escolhas certas. Certifique-se de incluir uma variedade de alimentos vegetais, como frutas, legumes, grãos integrais, leguminosas e oleaginosas. Consuma fontes de proteína vegetal, como feijões, lentilhas, tofu e quinoa, para garantir uma ingestão adequada de nutrientes. Além disso, considere suplementar com vitamina B12, que é encontrada principalmente em produtos de origem animal. Com um planejamento adequado, você pode desfrutar de todos os benefícios de uma dieta vegana."
}

# Documentos Veganos
documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [11]:
# Criação do DataFrame com os Novos Documentos Veganos
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]

In [12]:
# Carregamento do Modelo de Embedding
model = "models/embedding-001"

In [13]:
# Função para Incorporar Documentos
def embed_fn(title, text):
    return genai.embed_content(model=model,
                               content=text,
                               title=title,
                               task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [15]:
# Incorporação dos Documentos no DataFrame
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Receita Vegana: Hambúrguer de Grão-de-Bico,Experimente esta deliciosa receita vegana de h...,"[0.029435514, -0.039037086, -0.040710405, 0.00..."
1,Guia Vegano: Alternativas Saborosas aos Produt...,"Neste guia vegano, você encontrará alternativa...","[0.010990837, -0.058989566, -0.01639644, -0.02..."
2,Dicas para uma Dieta Vegana Equilibrada,Seguir uma dieta vegana pode ser saudável e eq...,"[-0.013332691, -0.06300287, 0.015886724, -0.01..."


In [16]:
# Função para Buscar e Retornar Trecho de Texto Relevante
def gerar_e_buscar_consulta(consulta, base, model):
    embedding_da_consulta = genai.embed_content(model=model,
                                                 content=consulta,
                                                 task_type="RETRIEVAL_QUERY")["embedding"]
    produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)
    indice = np.argmax(produtos_escalares)
    return df.iloc[indice]["Conteudo"]

In [18]:
# Consulta e Impressão do Trecho Relevante
consulta = "Como faço para ter uma dieta saudavel?"
trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Seguir uma dieta vegana pode ser saudável e equilibrado com as escolhas certas. Certifique-se de incluir uma variedade de alimentos vegetais, como frutas, legumes, grãos integrais, leguminosas e oleaginosas. Consuma fontes de proteína vegetal, como feijões, lentilhas, tofu e quinoa, para garantir uma ingestão adequada de nutrientes. Além disso, considere suplementar com vitamina B12, que é encontrada principalmente em produtos de origem animal. Com um planejamento adequado, você pode desfrutar de todos os benefícios de uma dieta vegana.


In [19]:
# Configuração de Geração de Texto
generation_config = {
    "temperature": 1,
    "candidate_count": 1
}

In [20]:
# Prompt para Geração de Texto
prompt = f"Você é um profissional de nutrição, explique de maneira clara e objetiva sem adicionar informações que não façam parte do texto: {trecho}"

In [21]:
# Inicialização do Modelo de Geração de Texto
model_2 = genai.GenerativeModel("gemini-1.0-pro", generation_config=generation_config)

In [22]:
# Geração de Texto mais criativo
response = model_2.generate_content(prompt)
print(response.text)

**Nutrição em uma Dieta Vegana**

Seguir uma dieta vegana pode ser saudável com escolhas alimentares adequadas. Inclua:

* **Variedade de Alimentos Vegetais:** Frutas, legumes, grãos integrais, leguminosas e oleaginosas.
* **Proteínas Vegetais:** Feijões, lentilhas, tofu, quinoa.
* **Vitamina B12:** Suplementação necessária, pois não é encontrada em alimentos vegetais.

Com planejamento adequado, uma dieta vegana pode fornecer todos os nutrientes essenciais.
